In [1]:
from __future__ import division
import os
import numpy as np
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import data_loader
from imp import reload
#reload(data_loader)

from data_loader import DataLoader
import module 
reload(module)
from module import *
from utils import *
import option
reload(option)
from option import *
#from keras import backend as K
from keras.callbacks import TensorBoard
#import tensorflow as tf 



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class cyclegan(object):
    def __init__(self, opt):
        self.discriminator = capsule_discriminator
        self.generator = generator_resnet
        self.criterionGAN = mae_criterion
        self.opt = opt
        self.img_shape = (self.opt.data_pix_size, self.opt.data_pix_size, self.opt.in_dim)
        self.build_model()
        self.pool = ImagePool(self.opt.max_size)

    def build_model(self):
        self.lambda_id = self.opt.lambda_id * self.opt.lambda_cycle
        self.d_A = capsule_discriminator(self.opt)
        self.d_B = capsule_discriminator(self.opt)
        self.optimizer = Adam(self.opt.lr, self.opt.beta1)
        self.d_A.compile(loss='binary_crossentropy',
                         optimizer=self.optimizer,
                         metrics=['accuracy'])
        self.d_B.compile(loss='binary_crossentropy',
                         optimizer=self.optimizer,
                         metrics=['accuracy'])
        self.g_AB = generator_unet()
        self.g_BA = generator_unet()
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)
        #style transfer
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        #reconstructure
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        #identity
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        #to train generator
        self.d_A.trainable = False
        self.d_B.trainable = False

        #dicriminator determine validity
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)
        

        # Combined model trains generators to fool discrgit/model.py:67iminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])


        self.combined.compile(loss=['mean_squared_error', 'mean_squared_error',
                                    'mean_squared_error', 'mean_squared_error',
                                    'mean_squared_error', 'mean_squared_error'],
                              loss_weights=[1, 1,
                                            self.opt.lambda_cycle, self.opt.lambda_cycle,
                                            self.lambda_id, self.lambda_id],
                              optimizer=self.optimizer)

        #callback.set_model(self.combined)
        #self.combined.load_weights('model.h5')

    def train(self):
        start_time = datetime.datetime.now()
        # Calculate output shape of D (PatchGAN)
        #patch = int(self.opt.d_patch_size)    ###### change it
        #self.disc_patch = (patch, patch, 1)   ###### change it

        # Adversarial loss ground truths
        #valid = np.ones((self.opt.batch_size,) + self.disc_patch)
        #fake = np.zeros((self.opt.batch_size,) + self.disc_patch)
        
        ######### for capsule ############
        valid = np.ones((self.opt.batch_size))
        fake = np.zeros((self.opt.batch_size,))
        
        
        self.data_loader = DataLoader(dataset_name=self.opt.dataset_name,
                                      img_res=(self.opt.data_pix_size, self.opt.data_pix_size))
        
        
        for epoch in range(25):
                for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(self.opt.batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                    fake_B = self.g_AB.predict_on_batch(imgs_A)
                    fake_A = self.g_BA.predict_on_batch(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                    dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                    dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                    dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                    dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                    dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                    dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                    d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                    g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                    elapsed_time = datetime.datetime.now() - start_time
                    #train_names = ['train_loss', 'train_mae']

                # Plot the progress
                    print (g_loss)
                    print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, self.opt.epochs,
                                                                            batch_i, self.data_loader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))
                    #write_log(callback, train_names, g_loss, batch_i)
                    
                    

                # If at save interval => save generated image samples
                    if batch_i % self.opt.sample_iter == 0:
                        self.sample_images(epoch, batch_i)

                #self.combined.save('model_unet_capsule.json')
                # serialize weights to HDF5
                self.combined.save_weights("weights_OCT_" + str(epoch) + '.h5')
                #print("Saved model to disk")




    def sample_images(self,epoch, batch_i):
        os.makedirs('images/%s' % self.opt.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.opt.dataset_name, epoch, batch_i))
        
        plt.close()


In [3]:
p = BaseOptions()
opt = p.args
gan1 = cyclegan(opt)
#gan.train()

In [4]:
opt

Namespace(batch_size=1, beta1=0.5, checkpoint_dir='./checkpoint', continue_train=False, d_fir_dim=64, d_patch_size=16, data_pix_size=256, dataset_name='infrared', epoch_step=100, epochs=25, g_fir_dim=32, in_dim=3, lambda_cycle=10.0, lambda_id=0.1, load_size=286, lr=0.0002, max_size=50, out_dim=3, phase='train', sample_dir='./sample', sample_iter=100, save_freq=1000, test_dir='./test', train_size=100000000.0, which_direction='AtoB')

In [10]:
gan.combined.load_weights('weights_unet_capsule_patches.h5')

In [4]:
gan1.train()

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[6.8464766, 0.24996501, 0.24997896, 0.06934643, 0.5060445, 0.07436715, 0.51825655]
[Epoch 0/25] [Batch 0/1080] [D loss: 0.693221, acc:  25%] [G loss: 6.846477, adv: 0.249972, recon: 0.287695, id: 0.074367] time: 0:00:17.870972 
[3.713711, 0.24995786, 0.24946265, 0.100421734, 0.18974197, 0.10503881, 0.20761451]
[Epoch 0/25] [Batch 1/1080] [D loss: 0.669963, acc:  50%] [G loss: 3.713711, adv: 0.249710, recon: 0.145082, id: 0.105039] time: 0:00:18.857450 
[5.23903, 0.24948904, 0.24487768, 0.08425597, 0.344765, 0.09033215, 0.3641216]
[Epoch 0/25] [Batch 2/1080] [D loss: 0.552084, acc:  50%] [G loss: 5.239030, adv: 0.247183, recon: 0.214510, id: 0.090332] time: 0:00:19.356603 
[6.678523, 0.24817248, 0.2442215, 0.20933795, 0.3506132, 0.2153709, 0.37124622]
[Epoch 0/25] [Batch 3/1080] [D loss: 0.437231, acc:  50%] [G loss: 6.678523, adv: 0.246197, recon: 0.279976, id: 0.215371] time: 0:00:19.846184 
[5.5437207, 0.23764545, 0.24211918, 0.10772332, 0.3482241, 0.11375666, 0.3907253]
[Epoch 0/25]

[3.4253662, 0.25931254, 1.0, 0.081118226, 0.11266707, 0.062491015, 0.1657098]
[Epoch 0/25] [Batch 36/1080] [D loss: 0.169311, acc: 100%] [G loss: 3.425366, adv: 0.629656, recon: 0.096893, id: 0.062491] time: 0:00:36.357586 
[3.4952922, 0.29642287, 1.0, 0.14078775, 0.05945795, 0.1104215, 0.08599089]
[Epoch 0/25] [Batch 37/1080] [D loss: 4.188569, acc:  75%] [G loss: 3.495292, adv: 0.648211, recon: 0.100123, id: 0.110422] time: 0:00:36.883795 
[2.2649317, 0.2535397, 1.0, 0.024471931, 0.06373429, 0.019427804, 0.109901994]
[Epoch 0/25] [Batch 38/1080] [D loss: 0.202485, acc: 100%] [G loss: 2.264932, adv: 0.626770, recon: 0.044103, id: 0.019428] time: 0:00:37.392570 
[1.9328228, 0.25791562, 0.15775609, 0.05671876, 0.08004492, 0.041931123, 0.10758312]
[Epoch 0/25] [Batch 39/1080] [D loss: 0.186858, acc:  75%] [G loss: 1.932823, adv: 0.207836, recon: 0.068382, id: 0.041931] time: 0:00:37.888764 
[3.1997554, 0.2466949, 0.9958656, 0.05625438, 0.11708322, 0.039444454, 0.18437457]
[Epoch 0/25] [B

[1.1166111, 0.2860672, 0.0, 0.051824942, 0.023264043, 0.044259313, 0.03539484]
[Epoch 0/25] [Batch 73/1080] [D loss: 4.421207, acc:  50%] [G loss: 1.116611, adv: 0.143034, recon: 0.037544, id: 0.044259] time: 0:00:55.119043 
[1.161002, 0.32079753, 0.0, 0.021037284, 0.05183199, 0.02039605, 0.0911158]
[Epoch 0/25] [Batch 74/1080] [D loss: 4.584938, acc:  50%] [G loss: 1.161002, adv: 0.160399, recon: 0.036435, id: 0.020396] time: 0:00:55.596494 
[0.94099647, 0.25942612, 0.0, 0.025012102, 0.033793837, 0.021785844, 0.07172515]
[Epoch 0/25] [Batch 75/1080] [D loss: 4.156044, acc:  75%] [G loss: 0.940996, adv: 0.129713, recon: 0.029403, id: 0.021786] time: 0:00:56.106409 
[1.7315793, 0.2758795, 0.0, 0.04543537, 0.08541363, 0.044060167, 0.10314974]
[Epoch 0/25] [Batch 76/1080] [D loss: 4.989428, acc:  50%] [G loss: 1.731579, adv: 0.137940, recon: 0.065425, id: 0.044060] time: 0:00:56.597922 
[0.8304743, 0.2672553, 0.0, 0.020501638, 0.03028849, 0.023673587, 0.0316441]
[Epoch 0/25] [Batch 77/108

[1.4902169, 0.32240015, 0.0, 0.06423198, 0.040754966, 0.052045744, 0.06590156]
[Epoch 0/25] [Batch 110/1080] [D loss: 4.158643, acc:  75%] [G loss: 1.490217, adv: 0.161200, recon: 0.052493, id: 0.052046] time: 0:01:14.291596 
[0.6381745, 0.25682402, 0.0, 0.01282653, 0.02076361, 0.0120917745, 0.0333573]
[Epoch 0/25] [Batch 111/1080] [D loss: 4.339944, acc:  50%] [G loss: 0.638174, adv: 0.128412, recon: 0.016795, id: 0.012092] time: 0:01:14.814992 
[0.8222168, 0.25316554, 0.0, 0.022880021, 0.028082715, 0.021022165, 0.03840175]
[Epoch 0/25] [Batch 112/1080] [D loss: 4.333477, acc:  50%] [G loss: 0.822217, adv: 0.126583, recon: 0.025481, id: 0.021022] time: 0:01:15.331551 
[0.9578431, 0.26964623, 0.0, 0.022445908, 0.039463624, 0.017343026, 0.051758535]
[Epoch 0/25] [Batch 113/1080] [D loss: 4.322060, acc:  75%] [G loss: 0.957843, adv: 0.134823, recon: 0.030955, id: 0.017343] time: 0:01:15.835048 
[0.96147954, 0.26430777, 0.0, 0.03581578, 0.026850406, 0.030611396, 0.03989854]
[Epoch 0/25] [

...

[1.2085779, 1.0, 0.0, 0.002179643, 0.015355457, 0.017062616, 0.016164158]
[Epoch 1/25] [Batch 371/1080] [D loss: 3.985596, acc:  75%] [G loss: 1.208578, adv: 0.500000, recon: 0.008768, id: 0.017063] time: 0:12:45.837087 
[1.213897, 1.0, 0.0, 0.004416981, 0.014076118, 0.01856787, 0.010398189]
[Epoch 1/25] [Batch 372/1080] [D loss: 3.985596, acc:  75%] [G loss: 1.213897, adv: 0.500000, recon: 0.009247, id: 0.018568] time: 0:12:46.346324 
[1.1693851, 1.0, 0.0, 0.0044838935, 0.01042261, 0.013754263, 0.00656577]
[Epoch 1/25] [Batch 373/1080] [D loss: 8.015120, acc:  50%] [G loss: 1.169385, adv: 0.500000, recon: 0.007453, id: 0.013754] time: 0:12:46.840596 
[0.17905058, 0.0, 0.0, 0.0048617944, 0.010484917, 0.010213632, 0.015369839]
[Epoch 1/25] [Batch 374/1080] [D loss: 7.887279, acc:  50%] [G loss: 0.179051, adv: 0.000000, recon: 0.007673, id: 0.010214] time: 0:12:47.374413 
[1.190404, 1.0, 0.0, 0.003747445, 0.011998694, 0.016181568, 0.01676118]
[Epoch 1/25] [Batch 375/1080] [D loss: 3.9855

[0.31988743, 0.0, 0.0, 0.010064267, 0.016976524, 0.03800083, 0.011478694]
[Epoch 1/25] [Batch 408/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.319887, adv: 0.000000, recon: 0.013520, id: 0.038001] time: 0:13:04.831648 
[0.3370233, 0.0, 0.0, 0.01087392, 0.019612694, 0.019050589, 0.013106545]
[Epoch 1/25] [Batch 409/1080] [D loss: 12.000715, acc:  25%] [G loss: 0.337023, adv: 0.000000, recon: 0.015243, id: 0.019051] time: 0:13:05.333298 
[0.3631671, 0.0, 0.0, 0.0055411267, 0.025175469, 0.019848537, 0.0361526]
[Epoch 1/25] [Batch 410/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.363167, adv: 0.000000, recon: 0.015358, id: 0.019849] time: 0:13:05.810106 
[0.18962014, 0.0, 0.0, 0.0041146055, 0.009829961, 0.03454338, 0.0156311]
[Epoch 1/25] [Batch 411/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.189620, adv: 0.000000, recon: 0.006972, id: 0.034543] time: 0:13:06.322147 
[0.09891241, 0.0, 0.0, 0.0043769027, 0.0035783309, 0.010508848, 0.008851222]
[Epoch 1/25] [Batch 412/1080] [D loss: 7.

[1.3583566, 1.0, 0.0, 0.0041114865, 0.027194064, 0.009540115, 0.035760984]
[Epoch 1/25] [Batch 445/1080] [D loss: 8.015120, acc:  50%] [G loss: 1.358357, adv: 0.500000, recon: 0.015653, id: 0.009540] time: 0:13:23.585299 
[0.13520026, 0.0, 0.0, 0.004243977, 0.006451321, 0.022629332, 0.0056179506]
[Epoch 1/25] [Batch 446/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.135200, adv: 0.000000, recon: 0.005348, id: 0.022629] time: 0:13:24.089615 
[1.2404631, 1.0, 0.0, 0.0067788223, 0.014054516, 0.014747613, 0.017382171]
[Epoch 1/25] [Batch 447/1080] [D loss: 8.015120, acc:  50%] [G loss: 1.240463, adv: 0.500000, recon: 0.010417, id: 0.014748] time: 0:13:24.590351 
[0.35117164, 0.0, 0.0, 0.01649654, 0.012984682, 0.029492345, 0.026867084]
[Epoch 1/25] [Batch 448/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.351172, adv: 0.000000, recon: 0.014741, id: 0.029492] time: 0:13:25.122750 
[0.37744996, 0.0, 0.0, 0.0051371693, 0.028187927, 0.01970011, 0.024498891]
[Epoch 1/25] [Batch 449/1080] [D loss

[0.37806493, 0.0, 0.0, 0.003675582, 0.028126214, 0.018194672, 0.041852303]
[Epoch 1/25] [Batch 482/1080] [D loss: 12.000715, acc:  25%] [G loss: 0.378065, adv: 0.000000, recon: 0.015901, id: 0.018195] time: 0:13:42.301236 
[0.6642243, 0.0, 0.0, 0.0049191597, 0.056517236, 0.012639739, 0.037220582]
[Epoch 1/25] [Batch 483/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.664224, adv: 0.000000, recon: 0.030718, id: 0.012640] time: 0:13:42.804670 
[0.16207135, 0.0, 0.0, 0.0040437044, 0.00915507, 0.021861102, 0.008222496]
[Epoch 1/25] [Batch 484/1080] [D loss: 7.971192, acc:  50%] [G loss: 0.162071, adv: 0.000000, recon: 0.006599, id: 0.021861] time: 0:13:43.288156 
[2.3996415, 1.0, 0.0, 0.017433323, 0.10547407, 0.04044634, 0.1301213]
[Epoch 1/25] [Batch 485/1080] [D loss: 11.683245, acc:  25%] [G loss: 2.399642, adv: 0.500000, recon: 0.061454, id: 0.040446] time: 0:13:43.778289 
[0.364964, 0.0, 0.0, 0.005242252, 0.027421834, 0.012890783, 0.025432348]
[Epoch 1/25] [Batch 486/1080] [D loss: 7.9

KeyboardInterrupt: 

In [35]:
a=generator_unet()

In [38]:
img_A = Input(shape=(256,256,3))

In [39]:
img_A

<tf.Tensor 'input_34:0' shape=(?, 256, 256, 3) dtype=float32>

In [40]:
a(img_A)

<tf.Tensor 'model_27/conv2d_144/Tanh:0' shape=(?, 256, 256, 3) dtype=float32>

In [60]:
gan.combined.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_57 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_45 (Model)                (None, 256, 256, 3)  1546819     input_58[0][0]                   
                                                                 model_44[1][0]                   
                                                                 input_57[0][0]                   
__________________________________________________________________________________________________
model_44 (

In [61]:
gan.d_A.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_53 (InputLayer)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_223 (Conv2D)          (None, 128, 128, 64)      3136      
_________________________________________________________________
leaky_re_lu_143 (LeakyReLU)  (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_224 (Conv2D)          (None, 64, 64, 128)       131200    
_________________________________________________________________
leaky_re_lu_144 (LeakyReLU)  (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_225 (Conv2D)          (None, 32, 32, 256)       524544    
_________________________________________________________________
leaky_re_lu_145 (LeakyReLU)  (None, 32, 32, 256)       0         
__________

C:\Users\Akshayaa\Anacond3\envs\py36\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [53]:
for a in gan.d_A.layers:
    print(a.trainable)

False
True
True
True
True
True
True
True
True
True


In [51]:
gan.d_A.trainable= False

In [52]:
gan.d_A.layers

In [7]:
opt.d_fir_dim

64